# Clinical Cohort Event Classification with Monte Carlo Cross-Validation

**Study Replication:** Publication-quality cohort-specific binary classification analysis with Monte Carlo cross-validation (configurable 50–100 splits)  
**Updated:** December 12, 2025  
**Hardware:** Optimized for 32-core EC2 instance (1TB RAM)  
**Validation:** Proper evaluation on unseen test data

## Key Features

✅ **Clinical Cohort Analysis** – CHD vs MyoCardio cohorts with modifiable clinical features  
✅ **Event Classification** – Binary classification at 1 year with AUC evaluation  
✅ **Monte Carlo Cross-Validation** – 50–100 random 80/20 train/test splits for cohort-level analysis  
✅ **Stratified Sampling** - Maintains event distribution  
✅ **Parallel Processing** - Fast execution with furrr/future (≈30 workers)  
✅ **95% Confidence Intervals** - Narrow, precise estimates (tighter with more splits)  
✅ **Realistic AUCs** - Expected range 0.70-0.90

## Methodology

This notebook implements binary classification for 1-year graft loss prediction:

1. Load data for pediatric heart transplant outcomes
2. Create binary outcome: event by 1 year vs. no event with ≥1 year follow-up
3. For each cohort and method:
   - Create 100–1000 stratified train/test splits (75/25)
   - Train classification model on training set
   - Evaluate on unseen test set
   - Aggregate results across splits
4. Calculate AUC, Brier Score, Accuracy, Precision, Recall, F1 with 95% CI
5. Extract top features

## Expected Runtime (per cohort × 4-method run)

- **100 splits (current default):**
  - Local (4 cores): ~1–2 hours (NOT RECOMMENDED)
  - Workstation (16 cores): ~30–60 minutes
  - EC2 (32 cores, 1TB RAM): ~30–60 minutes ✅ **RECOMMENDED FOR DEVELOPMENT**
- **1000 splits (extended / publication-level):**
  - Local (4 cores): 8–12+ hours (NOT RECOMMENDED)
  - Workstation (16 cores): ~4–8 hours
  - EC2 (32 cores, 1TB RAM): ~4–8 hours ✅ **RECOMMENDED FOR FINAL RESULTS**

## 1. Setup and Configuration

Load required packages and configure parallel processing.


In [ ]:
# Check R version
R.version.string

# Load required packages
library(here)
library(dplyr)
library(readr)
library(survival)
library(catboost)
library(tidyr)
library(purrr)
library(tibble)
library(janitor)
library(haven)
library(rsample)    # For MC-CV
library(furrr)      # For parallel processing
library(future)     # For parallel backend
library(progressr)  # For progress bars

# Load classification packages
library(glmnet)      # For LASSO
library(randomForest) # For Traditional RF
library(pROC)         # For AUC calculation
library(cutpointr)    # For threshold optimization

cat("✓ All packages loaded successfully\n")

# Source classification helper functions
source(here("scripts", "R", "classification_helpers.R"))
cat("✓ Classification helper functions loaded\n")

# Create output directory
output_dir <- here("cohort_analysis", "outputs", "classification")

# IDEMPOTENCY: Skip existing outputs if enabled
# Set SKIP_EXISTING_OUTPUTS = TRUE to resume from where you left off
# Set SKIP_EXISTING_OUTPUTS = FALSE to start fresh (will clean existing outputs)
SKIP_EXISTING_OUTPUTS <- TRUE  # Change to FALSE to force a fresh start

if (SKIP_EXISTING_OUTPUTS) {
  cat("\n")
  cat("╔════════════════════════════════════════════════════════════════╗\n")
  cat("║          🔄 RESUME MODE: Skipping existing outputs            ║\n")
  cat("╚════════════════════════════════════════════════════════════════╝\n")
  cat("Set SKIP_EXISTING_OUTPUTS = FALSE to force a fresh start\n\n")
} else {
  cat("\n")
  cat("╔════════════════════════════════════════════════════════════════╗\n")
  cat("║          🧹 FRESH START MODE: Cleaning existing outputs       ║\n")
  cat("╚════════════════════════════════════════════════════════════════╝\n\n")
  
  # Clean existing outputs directory to ensure fresh/clean results
  if (dir.exists(output_dir)) {
    # Remove all files in outputs directory
    output_files <- list.files(output_dir, full.names = TRUE, recursive = TRUE, include.dirs = FALSE)
    if (length(output_files) > 0) {
      cat(sprintf("Cleaning %d existing output files...\n", length(output_files)))
      file.remove(output_files)
    }
    # Remove empty subdirectories
    output_dirs <- list.dirs(output_dir, recursive = TRUE, full.names = TRUE)
    output_dirs <- output_dirs[output_dirs != output_dir]  # Don't remove main directory
    for (dir in rev(output_dirs)) {  # Reverse order to remove nested dirs first
      if (length(list.files(dir)) == 0) {
        unlink(dir, recursive = TRUE)
      }
    }
    cat("✓ Output directory cleaned\n")
  }
}

# Create output directory (if it doesn't exist)
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

cat("Output directory:", output_dir, "\n")
cat("MC-CV configuration will be displayed after setup is complete.\n")
flush.console()

In [ ]:
# ============================================================
# DEBUG/TEST MODE - Quick testing before full 100-split run
# ============================================================
# Set DEBUG_MODE = TRUE for quick testing (5 splits, ~2-5 min)
# Set DEBUG_MODE = FALSE for full analysis (100 splits, ~1-2 hours on EC2)
DEBUG_MODE <- FALSE  # Change to TRUE for quick test

# Analysis Mode: Event Classification (binary outcome at 1 year)
if (DEBUG_MODE) {  cat("\n")  cat("╔════════════════════════════════════════════════════════════════╗\n")  cat("║                    🔍 DEBUG MODE ENABLED                       ║\n")  cat("╚════════════════════════════════════════════════════════════════╝\n")  cat("\n")  cat("Quick test configuration:\n")  cat("  • MC-CV Splits: 5 (instead of 100)\n")  cat("  • Period: Original only (2010-2019)\n")  cat("  • Trees: Reduced (RSF: 100, AORSF: 50)\n")  cat("  • Expected time: 2-5 minutes\n")  cat("  • Purpose: Verify everything works before full run\n")  cat("\n")  cat("To run full analysis, set DEBUG_MODE = FALSE\n")  cat("\n")}# Configurationn_predictors <- 20                                        # Top 20 featuresn_trees_rsf <- if (DEBUG_MODE) 100 else 500              # RSF trees (reduced in debug)n_trees_aorsf <- if (DEBUG_MODE) 50 else 100             # AORSF trees (reduced in debug)horizon <- 1                                              # 1-year predictionn_mc_splits <- if (DEBUG_MODE) 5 else 100                # MC-CV splits (5 for debug, 100 for full)train_prop <- 0.75                                        # 75% training, 25% testing# Set up parallel processing - EC2 Optimized (32 cores, 1TB RAM)# Use 30 out of 32 cores (leave 2 for system)n_workers <- as.integer(Sys.getenv("N_WORKERS", "0"))if (n_workers < 1) {  total_cores <- parallel::detectCores()  n_workers <- max(1, total_cores - 2)  cat(sprintf("Auto-detected %d cores, using %d workers\n", total_cores, n_workers))}cat(sprintf("Setting up parallel processing with %d workers...\n", n_workers))cat(sprintf("Expected speedup: %dx faster than single core\n", round(n_workers * 0.8)))# Increase future.globals.maxSize for large MC-CV splits object# With 1TB RAM on EC2, we can handle large transfersoptions(future.globals.maxSize = 20 * 1024^3)  # 20 GB limit (plenty for 100 splits)cat("Set future.globals.maxSize to 20 GB\n")plan(multisession, workers = n_workers)

# Output directory is already created in Cell 2 with idempotency logic
# Just ensure it exists (don't clean it here - that's handled by SKIP_EXISTING_OUTPUTS in Cell 2)
output_dir <- here("cohort_analysis", "outputs", "classification")
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

cat("Output directory:", output_dir, "\n")
cat(sprintf("MC-CV Configuration: %d splits, %.0f/%.0f train/test split\n", 
            n_mc_splits, train_prop * 100, (1 - train_prop) * 100))
flush.console()

## 2. Helper Functions

Define functions for data preparation, C-index calculation, and prediction.


In [ ]:
# Prepare modeling data with leakage prevention# Supports both survival and classification modesprepare_modeling_data <- function(data, mode = "survival") {  if (mode == "survival") {    # Survival mode: Find time and status columns    time_col <- intersect(c("time", "outcome_int_graft_loss", "int_graft_loss"), names(data))[1]    status_col <- intersect(c("status", "outcome_graft_loss", "graft_loss"), names(data))[1]        if (is.na(time_col) || is.na(status_col)) {      stop("Cannot find time/status columns for survival mode")    }        # Rename to standard names    if (time_col != "time") data <- data %>% rename(time = !!time_col)    if (status_col != "status") data <- data %>% rename(status = !!status_col)  } else if (mode == "classification") {    # Classification mode: Find outcome column    outcome_col <- intersect(c("outcome", "outcome_1yr", "event_1yr"), names(data))[1]        if (is.na(outcome_col)) {      # Try to create outcome from time/status if available      if ("ev_time" %in% names(data) && "ev_type" %in% names(data)) {        data <- data %>%          mutate(            outcome = case_when(              ev_type == 1 & ev_time < 1 ~ 1L,  # Event by 1 year              ev_type == 0 & ev_time >= 1 ~ 0L,  # No event, follow-up >= 1 year              TRUE ~ NA_integer_  # Censored before 1 year - drop            )          ) %>%          filter(!is.na(outcome))  # Drop censored before 1 year        cat("→ Created outcome variable from ev_time and ev_type\n")      } else {        stop("Cannot find outcome column or ev_time/ev_type for classification mode")      }    } else if (outcome_col != "outcome") {      data <- data %>% rename(outcome = !!outcome_col)    }  } else {    stop("Invalid ANALYSIS_MODE. Must be 'survival' or 'classification'")  }    # Exclude leakage variables and identifier columns  exclude_exact <- c(    "ID", "ptid_e", "int_dead", "int_death", "graft_loss", "txgloss", "death", "event",    "dpricaus", "deathspc", "concod", "age_death", "dlist", "txpl_year",    "rrace_b", "rrace_a", "rrace_ai", "rrace_pi", "rrace_o", "rrace_un", "race",    "patsupp", "pmorexam", "papooth", "pacuref", "pishltgr",    "pathero", "pcadrec", "pcadrem", "pdiffib", "cpathneg",    "dcardiac", "dneuro", "dreject", "dsecaccs", "dpriaccs",    "dconmbld", "dconmal", "dconcard", "dconneur", "dconrej",    "dmajbld", "dmalcanc"  )    exclude_prefixes <- c("dtx_", "cc_", "dcon", "dpri", "dsec", "dmaj", "sd")    exclude_by_prefix <- character(0)  for (prefix in exclude_prefixes) {    exclude_by_prefix <- c(exclude_by_prefix,                            names(data)[startsWith(names(data), prefix)])  }    exclude_all <- unique(c(exclude_exact, exclude_by_prefix))  data <- data %>% select(-any_of(exclude_all))    # Median imputation for numeric variables  numeric_vars <- names(data)[sapply(data, is.numeric) & names(data) != "time" & names(data) != "status"]  for (var in numeric_vars) {    if (any(is.na(data[[var]]))) {      median_val <- median(data[[var]], na.rm = TRUE)      data[[var]][is.na(data[[var]])] <- median_val    }  }    # Mode imputation for categorical variables  categorical_vars <- names(data)[sapply(data, function(x) is.factor(x) | is.character(x))]  for (var in categorical_vars) {    if (any(is.na(data[[var]]))) {      mode_val <- names(sort(table(data[[var]]), decreasing = TRUE))[1]      data[[var]][is.na(data[[var]])] <- mode_val    }  }    # Remove constant columns  constant_cols <- names(data)[sapply(data, function(x) length(unique(na.omit(x))) <= 1)]  if (length(constant_cols) > 0) {    data <- data %>% select(-any_of(constant_cols))  }    # Convert character to factor  data <- data %>% mutate(across(where(is.character), as.factor))    return(data)}cat("✓ prepare_modeling_data() defined\n")

In [ ]:
# C-index calculation
calculate_cindex <- function(time, status, risk_scores, horizon = NULL) {
  valid_idx <- !is.na(time) & !is.na(status) & !is.na(risk_scores) &
               is.finite(time) & is.finite(risk_scores) & time > 0
  
  time   <- as.numeric(time[valid_idx])
  status <- as.numeric(status[valid_idx])
  risk   <- as.numeric(risk_scores[valid_idx])
  
  n <- length(time)
  events <- sum(status == 1)
  
  if (n < 10 || events < 1 || length(unique(risk)) == 1) {
    return(list(cindex_td = NA_real_, cindex_ti = NA_real_))
  }
  
  # Time-independent C-index (Harrell's)
  num_conc_ti <- 0
  num_disc_ti <- 0
  num_ties_ti <- 0
  
  for (i in seq_len(n)) {
    if (status[i] != 1) next
    for (j in seq_len(n)) {
      if (i == j) next
      if (time[i] < time[j]) {
        if (risk[i] > risk[j]) {
          num_conc_ti <- num_conc_ti + 1
        } else if (risk[i] < risk[j]) {
          num_disc_ti <- num_disc_ti + 1
        } else {
          num_ties_ti <- num_ties_ti + 1
        }
      }
    }
  }
  
  denom_ti <- num_conc_ti + num_disc_ti + num_ties_ti
  cindex_ti <- if (denom_ti > 0) (num_conc_ti + 0.5 * num_ties_ti) / denom_ti else NA_real_
  
  # Time-dependent C-index
  cindex_td <- tryCatch({
    score_data <- data.frame(time = time, status = status)
    pred_matrix <- matrix(risk, ncol = 1)
    
    evaluation <- riskRegression::Score(
      object = list(Model = pred_matrix),
      formula = Surv(time, status) ~ 1,
      data = score_data,
      times = if (!is.null(horizon)) horizon else median(time[status == 1]),
      summary = "risks",
      metrics = "auc",
      se.fit = FALSE
    )
    
    as.numeric(evaluation$AUC$score$AUC[1])
  }, error = function(e) cindex_ti)
  
  return(list(cindex_td = cindex_td, cindex_ti = cindex_ti))
}

# ranger_predictrisk function
ranger_predictrisk <- function(object, newdata, times) {
  preds <- predict(object, data = newdata, type = "response")
  if (is.null(preds$survival)) {
    stop("ranger prediction did not return survival probabilities")
  }
  
  surv_matrix <- preds$survival
  time_points <- preds$unique.death.times
  closest_idx <- which.min(abs(time_points - times))
  risk_scores <- 1 - surv_matrix[, closest_idx]
  
  return(as.numeric(risk_scores))
}

cat("✓ calculate_cindex() and ranger_predictrisk() defined\n")


## 3. Monte Carlo Cross-Validation Function

Main function that runs MC-CV for a single method and period.


In [ ]:
# Run MC-CV for a single method and time period
run_mc_cv_method <- function(data, method, period_name, mc_splits) {
  
  cat(sprintf("\n=== Running MC-CV for %s (%s) ===\n", method, period_name))
  cat(sprintf("Splits: %d | Train: %.0f%% | Test: %.0f%%\n", 
              n_mc_splits, train_prop * 100, (1 - train_prop) * 100))
  flush.console()
  
  split_ids <- seq_len(n_mc_splits)
  
  # Run splits in parallel with progress bar
  with_progress({
    p <- progressor(steps = n_mc_splits)
    
    results <- future_map(split_ids, function(split_id) {
      p()  # Update progress
      
      # Get train/test data from split
      split <- mc_splits$splits[[split_id]]
      train_data <- rsample::analysis(split)
      test_data <- rsample::assessment(split)
      
      # Train and evaluate
      model <- NULL
      predictions <- NULL
      feature_importance <- NULL
      
      tryCatch({
        if (method == "RSF") {
          model <- ranger(
            Surv(time, status) ~ .,
            data = train_data,
            num.trees = n_trees_rsf,
            importance = "permutation",
            min.node.size = 20,
            splitrule = "extratrees",
            num.random.splits = 10
          )
          
          predictions <- ranger_predictrisk(model, test_data, horizon)
          feature_importance <- model$variable.importance
          
        } else if (method == "AORSF") {
          # Remove constant columns from training data (can occur after train/test split)
          constant_cols <- names(train_data)[sapply(train_data, function(x) {
            if (is.numeric(x)) {
              length(unique(na.omit(x))) <= 1
            } else {
              length(unique(na.omit(x))) <= 1
            }
          })]
          constant_cols <- setdiff(constant_cols, c("time", "status"))
          if (length(constant_cols) > 0) {
            train_data <- train_data %>% select(-any_of(constant_cols))
            test_data <- test_data %>% select(-any_of(constant_cols))
          }
          
          model <- aorsf::orsf(
            data = train_data,
            formula = Surv(time, status) ~ .,
            n_tree = n_trees_aorsf,
            na_action = 'impute_meanmode'
          )
          
          pred_obj <- predict(model, new_data = test_data, 
                              pred_type = 'risk', pred_horizon = horizon)
          predictions <- if (is.matrix(pred_obj)) as.numeric(pred_obj[, 1]) else as.numeric(pred_obj)
          feature_importance <- aorsf::orsf_vi_permute(model)
          
        } else if (method == "CatBoost") {
          # Remove constant columns from training data (can occur after train/test split)
          constant_cols <- names(train_data)[sapply(train_data, function(x) {
            if (is.numeric(x)) {
              length(unique(na.omit(x))) <= 1
            } else {
              length(unique(na.omit(x))) <= 1
            }
          })]
          constant_cols <- setdiff(constant_cols, c("time", "status"))
          if (length(constant_cols) > 0) {
            train_data <- train_data %>% select(-any_of(constant_cols))
            test_data <- test_data %>% select(-any_of(constant_cols))
          }
          
          train_pool <- catboost.load_pool(
            data = train_data %>% select(-time, -status),
            label = train_data$time
          )
          
          test_pool <- catboost.load_pool(
            data = test_data %>% select(-time, -status)
          )
          
          # CatBoost configuration: single-threaded inside each R worker to avoid
          # logger/thread-safety issues, and quiet logging in parallel runs.
          params <- list(
            loss_function  = 'Cox',
            iterations     = 100,
            learning_rate  = 0.1,
            depth          = 6,
            thread_count   = 1,
            logging_level  = 'Silent',
            verbose        = 0L
          )
          
          model <- catboost.train(train_pool, params = params)
          predictions <- catboost.predict(model, test_pool)
          
          # Get feature importance - CatBoost returns a matrix with rownames as feature names
          # IMPORTANT: catboost.get_feature_importance() returns a matrix (not a named vector)
          # - Values are in the first column: importance_matrix[, 1]
          # - Feature names are in rownames: rownames(importance_matrix)
          # Convert to named vector for consistency with RSF and AORSF (which return named vectors directly)
          importance_matrix <- catboost.get_feature_importance(model)
          feature_importance <- as.numeric(importance_matrix[, 1])
          names(feature_importance) <- rownames(importance_matrix)
        }
        
        # Calculate C-index on TEST data
        cindex_result <- calculate_cindex(
          time = test_data$time,
          status = test_data$status,
          risk_scores = predictions,
          horizon = horizon
        )
        
        return(list(
          split_id = split_id,
          cindex_td = cindex_result$cindex_td,
          cindex_ti = cindex_result$cindex_ti,
          feature_importance = feature_importance,
          n_train = nrow(train_data),
          n_test = nrow(test_data),
          success = TRUE
        ))
        
      }, error = function(e) {
        return(list(
          split_id = split_id,
          cindex_td = NA_real_,
          cindex_ti = NA_real_,
          feature_importance = NULL,
          n_train = nrow(train_data),
          n_test = nrow(test_data),
          success = FALSE,
          error = e$message
        ))
      })
    }, .options = furrr_options(
      seed = TRUE,
      packages = c(
        "dplyr", "purrr", "tibble", "rsample",
        "ranger", "aorsf", "catboost",
        "riskRegression", "prodlim"
      )
    ))
  })
  
  # Aggregate results
  successful_splits <- Filter(function(x) x$success, results)
  n_successful <- length(successful_splits)
  
  cat(sprintf("Successful splits: %d / %d\n", n_successful, n_mc_splits))
  flush.console()
  
  # If all splits failed (e.g., CatBoost configuration/data issue),
  # do not stop the whole analysis. Instead, log a warning and return
  # an empty/NA summary so that other methods and periods can continue.
  if (n_successful == 0) {
    error_splits <- Filter(function(x) !is.null(x$error), results)
    if (length(error_splits) > 0) {
      first_error <- error_splits[[1]]$error
      cat(sprintf("WARNING: example error for %s (%s): %s\n",
                  method, period_name, first_error))
    }
    cat(sprintf("WARNING: All MC-CV splits failed for %s (%s). Skipping this method.\n",
                method, period_name))
    flush.console()
    
    return(list(
      method = method,
      period = period_name,
      n_splits = n_mc_splits,
      n_successful = 0,
      cindex_td_mean = NA_real_,
      cindex_td_sd = NA_real_,
      cindex_td_ci_lower = NA_real_,
      cindex_td_ci_upper = NA_real_,
      cindex_ti_mean = NA_real_,
      cindex_ti_sd = NA_real_,
      cindex_ti_ci_lower = NA_real_,
      cindex_ti_ci_upper = NA_real_,
      top_features = numeric(0)
    ))
  }
  
  # Extract C-indexes
  cindex_td_values <- sapply(successful_splits, function(x) x$cindex_td)
  cindex_ti_values <- sapply(successful_splits, function(x) x$cindex_ti)
  
  cindex_td_values <- cindex_td_values[!is.na(cindex_td_values)]
  cindex_ti_values <- cindex_ti_values[!is.na(cindex_ti_values)]
  
  # Aggregate feature importance
  # All methods (RSF, AORSF, CatBoost) return named numeric vectors
  all_feature_names <- unique(unlist(lapply(successful_splits, function(x) {
    if (is.null(x$feature_importance)) return(NULL)
    names(x$feature_importance)
  })))
  
  aggregated_importance <- sapply(all_feature_names, function(feature) {
    importances <- sapply(successful_splits, function(x) {
      if (is.null(x$feature_importance)) return(NA_real_)
      if (feature %in% names(x$feature_importance)) {
        return(as.numeric(x$feature_importance[feature]))
      }
      return(NA_real_)
    })
    mean(importances, na.rm = TRUE)
  })
  
  # Ensure aggregated_importance is a numeric vector
  aggregated_importance <- as.numeric(aggregated_importance)
  names(aggregated_importance) <- all_feature_names
  
  top_features <- sort(aggregated_importance, decreasing = TRUE)[1:min(n_predictors, length(aggregated_importance))]
  
  # Calculate statistics
  results_summary <- list(
    method = method,
    period = period_name,
    n_splits = n_mc_splits,
    n_successful = n_successful,
    cindex_td_mean = mean(cindex_td_values, na.rm = TRUE),
    cindex_td_sd = sd(cindex_td_values, na.rm = TRUE),
    cindex_td_ci_lower = quantile(cindex_td_values, 0.025, na.rm = TRUE),
    cindex_td_ci_upper = quantile(cindex_td_values, 0.975, na.rm = TRUE),
    cindex_ti_mean = mean(cindex_ti_values, na.rm = TRUE),
    cindex_ti_sd = sd(cindex_ti_values, na.rm = TRUE),
    cindex_ti_ci_lower = quantile(cindex_ti_values, 0.025, na.rm = TRUE),
    cindex_ti_ci_upper = quantile(cindex_ti_values, 0.975, na.rm = TRUE),
    top_features = top_features
  )
  
  # Print summary
  cat(sprintf("\n--- Results for %s (%s) ---\n", method, period_name))
  cat(sprintf("Time-Dependent C-Index: %.4f ± %.4f (95%% CI: %.4f - %.4f)\n",
              results_summary$cindex_td_mean,
              results_summary$cindex_td_sd,
              results_summary$cindex_td_ci_lower,
              results_summary$cindex_td_ci_upper))
  cat(sprintf("Time-Independent C-Index: %.4f ± %.4f (95%% CI: %.4f - %.4f)\n",
              results_summary$cindex_ti_mean,
              results_summary$cindex_ti_sd,
              results_summary$cindex_ti_ci_lower,
              results_summary$cindex_ti_ci_upper))
  # Display top 10 features sorted alphabetically for easier comparison
  top10_features <- names(top_features)[1:min(10, length(top_features))]
  top10_features_sorted <- sort(top10_features)
  cat(sprintf("Top 10 features (alphabetical): %s\n", paste(top10_features_sorted, collapse = ", ")))
  flush.console()
  
  return(results_summary)
}

cat("✓ run_mc_cv_method() defined\n")


## 4. Load Data

In [ ]:
# Find and load SAS file
sas_path_local <- here("data", "phts_txpl_ml.sas7bdat")
sas_path_external <- here("graft-loss-parallel-processing", "data", "phts_txpl_ml.sas7bdat")
sas_path_graft_loss <- here("graft-loss", "data", "phts_txpl_ml.sas7bdat")

sas_path <- NULL
for (path in c(sas_path_local, sas_path_external, sas_path_graft_loss)) {
  if (file.exists(path)) {
    sas_path <- path
    break
  }
}

if (is.null(sas_path)) {
  stop("Cannot find phts_txpl_ml.sas7bdat in any location")
}

cat("Loading data from:", sas_path, "\n")

# Load data
phts_base <- haven::read_sas(sas_path) %>%
  filter(TXPL_YEAR >= 2010) %>%
  janitor::clean_names() %>%
  rename(
    outcome_int_graft_loss = int_graft_loss,
    outcome_graft_loss = graft_loss
  ) %>%
  mutate(
    ID = 1:n(),
    across(.cols = where(is.character), ~ ifelse(.x %in% c("", "unknown", "missing"), NA_character_, .x)),
    across(.cols = where(is.character), as.factor),
    tx_mcsd = if ('txnomcsd' %in% names(.)) {
      if_else(txnomcsd == 'yes', 0, 1)
    } else if ('txmcsd' %in% names(.)) {
      txmcsd
    } else {
      NA_real_
    }
  )

cat(sprintf("✓ Loaded data: %d rows, %d columns\n", nrow(phts_base), ncol(phts_base)))


In [ ]:
# Define time periods
periods <- list()
periods$original <- phts_base %>% filter(txpl_year >= 2010 & txpl_year <= 2019)
periods$full <- phts_base %>% filter(txpl_year >= 2010)
periods$full_no_covid <- phts_base %>% filter(txpl_year >= 2010 & !(txpl_year >= 2020 & txpl_year <= 2023))

# Print summary
for (period_name in names(periods)) {
  period_data <- periods[[period_name]]
  n_events <- sum(period_data$outcome_graft_loss, na.rm = TRUE)
  event_rate <- 100 * n_events / nrow(period_data)
  
  cat(sprintf("Period: %s | N: %d | Events: %d (%.2f%%)\n", 
              period_name, nrow(period_data), n_events, event_rate))
}

cat("\n✓ Time periods defined\n")


## 5. Run Analysis

Execute MC-CV for all methods and periods.

- **100 splits (default full run):** Typically ~1–2 hours on a 32-core EC2 instance, longer on smaller machines.
- **1000 splits (extended run):** Linearly more expensive; expect roughly 8–10× the 100-split time.

**Note:** You can run each period separately by uncommenting only one `period_name` at a time, or reduce the number of methods/periods to shorten runtime further.


In [ ]:
# Select periods and methods to run
# In DEBUG_MODE, only run original period for speed
period_names <- if (DEBUG_MODE) {
  c("original")  # Debug: just one period (~2-5 min)
} else {
  c("original", "full", "full_no_covid")  # Full: all periods (~30-45 min)
}

# Methods to run. Include CatBoost (with single-threaded, quiet config)
# alongside RSF and AORSF.
method_names <- c("RSF", "CatBoost", "AORSF")

# Store all results
all_results <- list()

# Run analysis
for (period_name in period_names) {
  cat(sprintf("\n========================================\n"))
  cat(sprintf("Processing Period: %s\n", period_name))
  cat(sprintf("========================================\n"))
  flush.console()
  
  # Prepare data
  period_data <- prepare_modeling_data(periods[[period_name]])
  
  cat(sprintf("Period data: %d rows, %d columns\n", nrow(period_data), ncol(period_data)))
  cat(sprintf("Events: %d (%.2f%%)\n", sum(period_data$status), 
              100 * sum(period_data$status) / nrow(period_data)))
  
  # Create MC-CV splits (stratified by outcome)
  cat(sprintf("Creating %d MC-CV splits (stratified)...\n", n_mc_splits))
  flush.console()
  mc_splits <- mc_cv(
    data = period_data,
    prop = train_prop,
    times = n_mc_splits,
    strata = status
  )
  
  # Run each method
  period_results <- list()
  
  for (method in method_names) {
    result <- run_mc_cv_method(period_data, method, period_name, mc_splits)
    period_results[[method]] <- result
    
    # Save top features (sorted alphabetically for easier comparison)
    top_features_df <- tibble(
      feature = names(result$top_features),
      importance = as.numeric(result$top_features),
      cindex_td = result$cindex_td_mean,
      cindex_ti = result$cindex_ti_mean
    ) %>%
      arrange(feature)  # Sort alphabetically for easier visual comparison
    
    output_file <- file.path(output_dir, sprintf("%s_%s_top20.csv", 
                                                  period_name, tolower(method)))
    write_csv(top_features_df, output_file)
    cat(sprintf("✓ Saved: %s\n", basename(output_file)))
  }
  
  all_results[[period_name]] <- period_results
}

cat("\n✓ Analysis complete for all periods!\n")


## 6. Save Summary Results

Create summary tables with C-index comparisons and statistics.


In [ ]:
# Create C-index comparison table
cindex_comparison <- map_df(period_names, function(period) {
  map_df(method_names, function(method) {
    result <- all_results[[period]][[method]]
    tibble(
      period = period,
      method = method,
      cindex_td_mean = result$cindex_td_mean,
      cindex_td_sd = result$cindex_td_sd,
      cindex_td_ci_lower = result$cindex_td_ci_lower,
      cindex_td_ci_upper = result$cindex_td_ci_upper,
      cindex_ti_mean = result$cindex_ti_mean,
      cindex_ti_sd = result$cindex_ti_sd,
      cindex_ti_ci_lower = result$cindex_ti_ci_lower,
      cindex_ti_ci_upper = result$cindex_ti_ci_upper,
      n_splits = result$n_successful
    )
  })
})

write_csv(cindex_comparison, file.path(output_dir, "cindex_comparison_mc_cv.csv"))
cat("✓ Saved: cindex_comparison_mc_cv.csv\n")

# Display the table
print(cindex_comparison)


In [ ]:
# Create summary statistics
summary_stats <- map_df(period_names, function(period) {
  period_data <- periods[[period]]
  tibble(
    period = period,
    n_patients = nrow(period_data),
    n_events = sum(period_data$outcome_graft_loss, na.rm = TRUE),
    event_rate = 100 * sum(period_data$outcome_graft_loss, na.rm = TRUE) / nrow(period_data)
  )
})

write_csv(summary_stats, file.path(output_dir, "summary_statistics_mc_cv.csv"))
cat("✓ Saved: summary_statistics_mc_cv.csv\n")

# Display the table
print(summary_stats)


In [ ]:
# Close parallel processing
plan(sequential)

cat("\n========================================\n")
cat("Analysis Complete!\n")
cat("========================================\n")
cat(sprintf("Output directory: %s\n", output_dir))
cat(sprintf("MC-CV splits: %d\n", n_mc_splits))
cat(sprintf("Train/Test ratio: %.0f/%.0f\n", train_prop * 100, (1 - train_prop) * 100))
cat("\nResults show C-indexes with 95% confidence intervals\n")
cat("based on", n_mc_splits, "independent train/test splits.\n\n")
cat("✓ All files saved successfully!\n")


## 7. Visualize Results

In [ ]:
# Create visualization plots for feature importance and C-index analysis
# Uses the updated create_visualizations.R script with improved normalization procedure
# Creates: Feature importance heatmap, C-index heatmap, scaled bar chart, and C-index table

library(here)

# Source the visualization script from scripts/R/
# Scripts are consolidated in scripts/R/ to match EC2 structure
if (!file.exists(here("scripts", "R", "create_visualizations_cohort.R"))) {
  stop("Cannot find scripts/R/create_visualizations_cohort.R. Ensure scripts are in scripts/R/ directory.")
}
source(here("scripts", "R", "create_visualizations_cohort.R"))

# Run visualizations
# Pass the output directory to ensure visualizations are saved to the correct location
run_visualizations(output_dir = output_dir)


## 7. Sync Results and Code to S3

Sync output files and code (notebook + R script) to S3 bucket after all analyses and visualizations are complete.
- Outputs: CSV results files, plots, and visualizations
- Code: Notebook and R script for reproducibility

In [ ]:
# Sync outputs and code to S3
# On EC2, we're in the cohort_analysis directory
s3_bucket <- "s3://uva-private-data-lake/graft-loss/cohort_analysis/classification/"

# Ensure output_dir_base is defined (should already exist from Section 6.6, but define if needed)
if (!exists("output_dir_base")) {
  output_dir_base <- here("cohort_analysis", "outputs", "classification")
}

# Find AWS CLI (check common locations - EC2 typically has it in /usr/local/bin or /usr/bin)
aws_cmd <- Sys.which("aws")
if (aws_cmd == "") {
  # Try common EC2 installation paths
  aws_paths <- c(
    "/usr/local/bin/aws",
    "/usr/bin/aws",
    "/home/ec2-user/.local/bin/aws"
  )
  aws_cmd <- NULL
  for (path in aws_paths) {
    if (file.exists(path)) {
      aws_cmd <- path
      break
    }
  }
  if (is.null(aws_cmd)) {
    stop("AWS CLI not found. Please install AWS CLI or ensure it's in your PATH.")
  }
}

cat("Syncing outputs and code to S3...\n")
cat("Source: cohort_analysis/ directory\n")
cat("Destination:", s3_bucket, "\n")
cat("AWS CLI:", aws_cmd, "\n\n")

# Get current directory (should be cohort_analysis)
current_dir <- getwd()
if (!grepl("cohort_analysis", current_dir)) {
  warning("Current directory doesn't appear to be cohort_analysis. Double-check sync destination.")
}

# Sync classification outputs and code to S3
# Sync the outputs/classification directory specifically to avoid overwriting survival outputs
# Explicitly include notebook, R scripts, README files, and outputs/classification directory
# Exclude temporary files, checkpoints, and unnecessary directories
# Note: --delete flag removed for safety (won't delete files in S3 that don't exist locally)
sync_cmd <- sprintf(
  '"%s" s3 sync "%s" %s --include "*.ipynb" --include "*.R" --include "README*.md" --include "outputs/classification/**" --exclude "*checkpoint*" --exclude "*.tmp" --exclude "*.ipynb_checkpoints/*" --exclude "*.RData" --exclude "*.Rhistory" --exclude ".Rproj.user/*" --exclude "catboost_info/*" --exclude "*.log" --exclude "*"',
  aws_cmd,
  current_dir,
  s3_bucket
)

cat("Running:", sync_cmd, "\n\n")
result <- system(sync_cmd)

if (result == 0) {
  cat("✓ Successfully synced outputs and code to S3\n")
  cat("  - Outputs:", file.path(output_dir_base), "\n")
  cat("  - Code: *.ipynb, *.R, README*.md\n")
} else {
  warning(sprintf("S3 sync returned exit code %d. Check AWS credentials and permissions.", result))
}


## 8. Shutdown EC2

Shutdown EC2 instance after all analyses, visualizations, and S3 sync are complete.

In [ ]:

# Shutdown EC2 instance after analysis completes
# Set SHUTDOWN_EC2 = TRUE to enable, FALSE to disable
SHUTDOWN_EC2 <- TRUE  # Change to TRUE to enable auto-shutdown

if (SHUTDOWN_EC2) {
  cat("\n========================================\n")
  cat("Shutting down EC2 instance...\n")
  cat("========================================\n")
  
  # Get instance ID from EC2 metadata service
  instance_id <- tryCatch({
    system("curl -s http://169.254.169.254/latest/meta-data/instance-id", intern = TRUE)
  }, error = function(e) {
    cat("Warning: Could not retrieve instance ID from metadata service.\n")
    cat("If running on EC2, check that metadata service is accessible.\n")
    return(NULL)
  })
  
  if (!is.null(instance_id) && length(instance_id) > 0 && nchar(instance_id[1]) > 0) {
    instance_id <- instance_id[1]
    cat(sprintf("Instance ID: %s\n", instance_id))
    
    # Find AWS CLI
    aws_cmd <- Sys.which("aws")
    if (aws_cmd == "") {
      aws_paths <- c(
        "/usr/local/bin/aws",
        "/usr/bin/aws",
        "/home/ec2-user/.local/bin/aws"
      )
      aws_cmd <- NULL
      for (path in aws_paths) {
        if (file.exists(path)) {
          aws_cmd <- path
          break
        }
      }
    }
    
    if (!is.null(aws_cmd) && aws_cmd != "") {
      # Stop the instance (use terminate-instances for permanent deletion)
      shutdown_cmd <- sprintf(
        '"%s" ec2 stop-instances --instance-ids %s',
        aws_cmd,
        instance_id
      )
      
      cat("Running:", shutdown_cmd, "\n")
      result <- system(shutdown_cmd)
      
      if (result == 0) {
        cat("✓ EC2 instance stop command sent successfully\n")
        cat("Instance will stop in a few moments.\n")
        cat("Note: This is a STOP (not terminate), so you can restart it later.\n")
      } else {
        warning(sprintf("EC2 stop command returned exit code %d. Check AWS credentials and permissions.", result))
      }
    } else {
      cat("Warning: AWS CLI not found. Cannot shutdown instance.\n")
      cat("Install AWS CLI or ensure it's in your PATH.\n")
    }
  } else {
    cat("Warning: Could not determine instance ID. Skipping shutdown.\n")
    cat("If you want to shutdown manually, use:\n")
    cat("  aws ec2 stop-instances --instance-ids <your-instance-id>\n")
  }
} else {
  cat("\n========================================\n")
  cat("EC2 Auto-Shutdown: DISABLED\n")
  cat("========================================\n")
  cat("To enable auto-shutdown, set SHUTDOWN_EC2 = TRUE in this cell.\n")
  cat("Instance will continue running.\n")
}


## 6. Clinical Feature Importance by Cohort (Modifiable Features)

This section implements binary classification models for 1-year event prediction.

**Models:**
- LASSO (logistic regression)
- CatBoost (classification)
- CatBoost RF (classification)
- Traditional RF (classification)

**Metrics:** AUC, Brier Score, Accuracy, Precision, Recall, F1


In [ ]:
cat("→ Starting Classification Analysis Mode\n\n")

# 6.1 Define classification lagging keywords (variables to exclude)
classification_lagging_keywords <- c(
  # Identifiers and outcomes
  "transplant_year", "primary_etiology",
  
  # Time variables (not needed for classification)
  "ev_", "days_to_last_followup", "time", "status",
  
  # Donor-specific variables
  "graft_loss", "int_graft_loss", "dtx_", "cc_", "isc_oth",
  "dcardiac", "dcon", "dpri", "dpricaus", "rec_", "papooth",
  "dneuro", "sdprathr", "int_dead", "listing_year", "cpathneg",
  "dcauseod",
  
  # Demographics (if not clinically relevant)
  "race", "sex", "drace_b", "rrace_a", "hisp", "lscntry",
  
  # Transplant-specific variables
  "dreject", "dsecaccsEmpty", "dmajbldEmpty", "pishltgr1R", 
  "drejectEmpty", "drejectHyperacute", "pishltgrEmpty",
  "pishltgr", "dmajbld", "dsecaccs", "dsecaccs_bin", 
  
  # Clinical variables to exclude
  "dx_cardiomyopathy", "deathspc", "dlist", "pmorexam", 
  "patsupp", "concod", "pcadrem", "pcadrec", "pathero", 
  "pdiffib", "dmalcanc", "alt_tx", "age_death", "pacuref",
  "cpbypass",
  
  # Additional variables
  "lsvcma"
)

cat("✓ Classification lagging keywords defined\n")


## 6.2 Define Cohorts and Prepare Data

Define cohorts (CHD vs MyoCardio) and prepare data for classification analysis.


In [ ]:
# 6.2 Define Cohorts and Prepare Data
# Prepare data for classification (ensure outcome variable exists)
# Use phts_base loaded earlier in the notebook and create outcome variable

cat("→ Preparing cohorts for classification analysis\n")

# Create outcome variable: event by 1 year vs. no event with ≥1 year follow-up
# Use ev_time and ev_type if available, otherwise use outcome_int_graft_loss and outcome_graft_loss
if ("ev_time" %in% names(phts_base) && "ev_type" %in% names(phts_base)) {
  phts_base <- phts_base %>%
    mutate(
      outcome = case_when(
        ev_type == 1 & ev_time < 1 ~ 1L,  # Event by 1 year
        ev_type == 0 & ev_time >= 1 ~ 0L,  # No event, follow-up >= 1 year
        TRUE ~ NA_integer_  # Censored before 1 year - will be dropped
      )
    ) %>%
    filter(!is.na(outcome))  # Drop censored before 1 year
  cat("→ Created outcome variable from ev_time and ev_type\n")
} else if ("outcome_int_graft_loss" %in% names(phts_base) && "outcome_graft_loss" %in% names(phts_base)) {
  # Fallback: use time-to-event data to create binary outcome
  phts_base <- phts_base %>%
    mutate(
      outcome = case_when(
        outcome_graft_loss == 1 & outcome_int_graft_loss < 1 ~ 1L,  # Event by 1 year
        outcome_graft_loss == 0 & outcome_int_graft_loss >= 1 ~ 0L,  # No event, follow-up >= 1 year
        TRUE ~ NA_integer_
      )
    ) %>%
    filter(!is.na(outcome))
  cat("→ Created outcome variable from outcome_int_graft_loss and outcome_graft_loss\n")
} else {
  stop("Cannot find ev_time/ev_type or outcome_int_graft_loss/outcome_graft_loss to create outcome variable")
}

# Split into cohorts
chd_data <- phts_base %>% filter(primary_etiology == "Congenital HD")
mc_data <- phts_base %>% filter(primary_etiology %in% c("Cardiomyopathy", "Myocarditis"))

cat("CHD Cohort Size:", nrow(chd_data), "patients\n")
cat("MC Cohort Size:", nrow(mc_data), "patients\n")
cat("CHD Event Rate:", round(mean(chd_data$outcome, na.rm = TRUE) * 100, 2), "%\n")
cat("MC Event Rate:", round(mean(mc_data$outcome, na.rm = TRUE) * 100, 2), "%\n\n")

# Prepare data using prepare_modeling_data (classification mode)
chd_data_prep <- prepare_modeling_data(chd_data, mode = "classification")
mc_data_prep <- prepare_modeling_data(mc_data, mode = "classification")

cat("✓ Data prepared for classification analysis\n")


In [ ]:
# 6.3 Classification Models with MC-CV
# This section implements MC-CV for classification models similar to survival models

cat("→ Starting Classification MC-CV Analysis\n")
cat("→ Number of MC-CV splits:", n_mc_splits, "\n\n")

# Classification model wrapper functions
fit_lasso_classification <- function(train_data, test_data) {
  # Prepare data
  y_train <- train_data$outcome
  x_train <- as.matrix(train_data %>% select(-outcome))
  y_test <- test_data$outcome
  x_test <- as.matrix(test_data %>% select(-outcome))
  
  # Fit LASSO
  set.seed(1997)
  cv_fit <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 1, nfolds = 5)
  pred_probs <- predict(cv_fit, newx = x_test, s = "lambda.min", type = "response")[, 1]
  
  # Calculate metrics
  metrics <- create_classification_metrics(pred_probs, y_test, "", "LASSO")
  return(list(metrics = metrics, predictions = pred_probs, actual = y_test))
}

fit_catboost_classification <- function(train_data, test_data) {
  # Prepare data
  y_train <- train_data$outcome
  x_train <- train_data %>% select(-outcome)
  y_test <- test_data$outcome
  x_test <- test_data %>% select(-outcome)
  
  # Convert to CatBoost format
  train_pool <- catboost.load_pool(data = x_train, label = y_train)
  test_pool <- catboost.load_pool(data = x_test, label = y_test)
  
  # Fit CatBoost
  set.seed(1997)
  params <- list(
    iterations = if (DEBUG_MODE) 100 else 500,
    learning_rate = 0.1,
    depth = 6,
    loss_function = "Logloss",
    verbose = FALSE
  )
  model <- catboost.train(train_pool, params = params)
  pred_probs <- catboost.predict(model, test_pool, prediction_type = "Probability")
  
  # Calculate metrics
  metrics <- create_classification_metrics(pred_probs, y_test, "", "CatBoost")
  return(list(metrics = metrics, predictions = pred_probs, actual = y_test))
}

fit_catboost_rf_classification <- function(train_data, test_data) {
  # Prepare data
  y_train <- train_data$outcome
  x_train <- train_data %>% select(-outcome)
  y_test <- test_data$outcome
  x_test <- test_data %>% select(-outcome)
  
  # Convert to CatBoost format
  train_pool <- catboost.load_pool(data = x_train, label = y_train)
  test_pool <- catboost.load_pool(data = x_test, label = y_test)
  
  # Fit CatBoost RF
  set.seed(1997)
  params <- list(
    iterations = if (DEBUG_MODE) 100 else 500,
    depth = 8,
    learning_rate = 1.0,
    bootstrap_type = "Bernoulli",
    subsample = 0.8,
    rsm = 0.5,
    loss_function = "Logloss",
    verbose = FALSE
  )
  model <- catboost.train(train_pool, params = params)
  pred_probs <- catboost.predict(model, test_pool, prediction_type = "Probability")
  
  # Calculate metrics
  metrics <- create_classification_metrics(pred_probs, y_test, "", "CatBoost RF")
  return(list(metrics = metrics, predictions = pred_probs, actual = y_test))
}

fit_rf_classification <- function(train_data, test_data) {
  # Prepare data
  y_train <- as.factor(train_data$outcome)
  x_train <- train_data %>% select(-outcome)
  y_test <- test_data$outcome
  x_test <- test_data %>% select(-outcome)
  
  # Fit Random Forest
  set.seed(1997)
  model <- randomForest(x = x_train, y = y_train, ntree = if (DEBUG_MODE) 100 else 500)
  pred_probs <- predict(model, x_test, type = "prob")[, 2]
  
  # Calculate metrics
  metrics <- create_classification_metrics(pred_probs, y_test, "", "Traditional RF")
  return(list(metrics = metrics, predictions = pred_probs, actual = y_test))
}

cat("✓ Classification model wrapper functions defined\n")


In [ ]:
# 6.4 Run MC-CV for classification models
# Similar structure to survival MC-CV but with classification metrics

cat("→ Running MC-CV for classification models\n")

# Prepare data for each cohort (remove lagging keywords)
prepare_classification_data <- function(data) {
  data %>%
    select(!matches(paste(classification_lagging_keywords, collapse = "|"))) %>%
    select(-ptid_e) %>%
    mutate(across(where(is.numeric), ~if_else(is.infinite(.), NA_real_, .))) %>%
    select(-where(~all(is.na(.)))) %>%
    mutate(across(where(is.character), as.factor)) %>%
    mutate(across(where(is.factor), as.numeric)) %>%
    filter(!is.na(outcome))  # Remove any missing outcomes
}

chd_class_data <- prepare_classification_data(chd_data_prep)
mc_class_data <- prepare_classification_data(mc_data_prep)

cat("CHD classification data:", nrow(chd_class_data), "patients\n")
cat("MC classification data:", nrow(mc_class_data), "patients\n\n")

# MC-CV function for classification
run_classification_mc_cv <- function(data, cohort_name, n_splits = n_mc_splits) {
  cat("→ Running MC-CV for", cohort_name, "cohort\n")
  
  # Remove constant columns
  constant_cols <- names(data)[sapply(data, function(x) length(unique(na.omit(x))) <= 1)]
  if (length(constant_cols) > 0) {
    data <- data %>% select(-any_of(constant_cols))
  }
  
  # Impute missing values
  for (var in names(data)) {
    if (is.numeric(data[[var]])) {
      data[[var]][is.na(data[[var]])] <- median(data[[var]], na.rm = TRUE)
    } else if (is.factor(data[[var]])) {
      mode_val <- names(sort(table(data[[var]]), decreasing = TRUE))[1]
      data[[var]][is.na(data[[var]])] <- mode_val
    }
  }
  
  # Store results
  all_metrics <- list()
  
  # Run MC-CV splits
  with_progress({
    p <- progressor(steps = n_splits)
    
    results <- future_map(1:n_splits, function(split_idx) {
      p()
      
      # Create train/test split
      set.seed(1997 + split_idx)
      split <- initial_split(data, prop = train_prop, strata = outcome)
      train_data <- training(split)
      test_data <- testing(split)
      
      # Fit models and collect metrics
      metrics_list <- list()
      
      tryCatch({
        lasso_result <- fit_lasso_classification(train_data, test_data)
        metrics_list$LASSO <- lasso_result$metrics %>% mutate(Cohort = cohort_name)
      }, error = function(e) {
        cat("Error in LASSO for split", split_idx, ":", e$message, "\n")
      })
      
      tryCatch({
        catboost_result <- fit_catboost_classification(train_data, test_data)
        metrics_list$CatBoost <- catboost_result$metrics %>% mutate(Cohort = cohort_name)
      }, error = function(e) {
        cat("Error in CatBoost for split", split_idx, ":", e$message, "\n")
      })
      
      tryCatch({
        catboost_rf_result <- fit_catboost_rf_classification(train_data, test_data)
        metrics_list$`CatBoost RF` <- catboost_rf_result$metrics %>% mutate(Cohort = cohort_name)
      }, error = function(e) {
        cat("Error in CatBoost RF for split", split_idx, ":", e$message, "\n")
      })
      
      tryCatch({
        rf_result <- fit_rf_classification(train_data, test_data)
        metrics_list$`Traditional RF` <- rf_result$metrics %>% mutate(Cohort = cohort_name)
      }, error = function(e) {
        cat("Error in Traditional RF for split", split_idx, ":", e$message, "\n")
      })
      
      return(metrics_list)
    }, .options = furrr_options(seed = TRUE))
  })
  
  # Aggregate metrics across splits
  all_metrics_df <- bind_rows(lapply(results, function(x) bind_rows(x)))
  
  # Calculate summary statistics
  summary_metrics <- all_metrics_df %>%
    group_by(Cohort, Model) %>%
    summarise(
      AUC_mean = mean(AUC, na.rm = TRUE),
      AUC_sd = sd(AUC, na.rm = TRUE),
      AUC_ci_lower = AUC_mean - 1.96 * AUC_sd / sqrt(n()),
      AUC_ci_upper = AUC_mean + 1.96 * AUC_sd / sqrt(n()),
      Brier_mean = mean(Brier_Score, na.rm = TRUE),
      Brier_sd = sd(Brier_Score, na.rm = TRUE),
      Accuracy_mean = mean(Accuracy, na.rm = TRUE),
      Precision_mean = mean(Precision, na.rm = TRUE),
      Recall_mean = mean(Recall, na.rm = TRUE),
      F1_mean = mean(F1, na.rm = TRUE),
      n_splits = n(),
      .groups = "drop"
    )
  
  return(list(
    all_metrics = all_metrics_df,
    summary = summary_metrics
  ))
}

# Run MC-CV for both cohorts
cat("→ Starting MC-CV for CHD cohort\n")
chd_mc_cv_results <- run_classification_mc_cv(chd_class_data, "CHD", n_mc_splits)

cat("\n→ Starting MC-CV for MC cohort\n")
mc_mc_cv_results <- run_classification_mc_cv(mc_class_data, "MC", n_mc_splits)

# Combine results
classification_mc_metrics <- bind_rows(
  chd_mc_cv_results$summary,
  mc_mc_cv_results$summary
)

cat("\n✓ Classification MC-CV completed\n")
cat("→ Results summary:\n")
print(classification_mc_metrics)


## 6.5 Normalized Scaled Feature Importance Bar Chart

Create a normalized scaled feature importance bar chart showing the top 25 modifiable clinical features. Importance is normalized within each cohort/model and scaled by model performance (MC-CV AUC).


In [ ]:
# 6.5 Normalized Scaled Feature Importance Bar Chart

# Create normalized scaled feature importance bar chart for top clinical features
# This chart shows feature importance normalized within each cohort/model and scaled by model performance (AUC)

library(ggplot2)
library(dplyr)

# Check if required files exist
output_dir_base <- here("cohort_analysis", "outputs", "classification")
summary_dir <- file.path(output_dir_base, "summary")

best_feat_path <- file.path(summary_dir, "best_clinical_features_by_cohort_mc_cv.csv")
metrics_path <- file.path(summary_dir, "cohort_classification_metrics_mc_cv.csv")

if (file.exists(best_feat_path) && file.exists(metrics_path)) {
  # Read data
  best_features <- readr::read_csv(best_feat_path, show_col_types = FALSE)
  classification_metrics <- readr::read_csv(metrics_path, show_col_types = FALSE)
  
  # Prepare feature matrix: Cohort × Model × Feature
  feature_matrix <- best_features %>%
    dplyr::select(Cohort, Model, feature, importance) %>%
    dplyr::mutate(
      Cohort = as.character(Cohort),
      Model  = as.character(Model),
      importance = as.numeric(importance)
    ) %>%
    dplyr::mutate(importance = ifelse(is.na(importance), 0, importance)) %>%
    dplyr::group_by(Cohort, Model) %>%
    dplyr::mutate(
      importance = ifelse(importance < 0, 0, importance),
      total_imp = sum(importance),
      importance_normalized = ifelse(total_imp > 0, importance / total_imp, 1 / dplyr::n())
    ) %>%
    dplyr::select(-total_imp) %>%
    dplyr::ungroup()
  
  # Relative weights per cohort/model using AUC (instead of C-index for classification)
  algorithm_ranking <- classification_metrics %>%
    dplyr::select(Cohort, Model, AUC_mean) %>%
    dplyr::group_by(Cohort) %>%
    dplyr::mutate(
      best_auc = max(AUC_mean, na.rm = TRUE),
      n_models = sum(!is.na(AUC_mean))
    ) %>%
    dplyr::ungroup() %>%
    dplyr::mutate(
      rel_weight = ifelse(best_auc > 0,
                          (AUC_mean / best_auc) * n_models,
                          1)
    ) %>%
    dplyr::select(Cohort, Model, rel_weight)
  
  feature_matrix <- feature_matrix %>%
    dplyr::left_join(algorithm_ranking, by = c("Cohort", "Model")) %>%
    dplyr::mutate(
      rel_weight = ifelse(is.na(rel_weight), 1, rel_weight),
      importance_scaled = importance_normalized * rel_weight
    )
  
  # Create scaled feature importance bar chart (Top 25 features)
  scaled_feature_importance <- feature_matrix %>%
    dplyr::group_by(feature) %>%
    dplyr::summarise(
      total_scaled_importance = sum(importance_scaled, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    dplyr::arrange(desc(total_scaled_importance)) %>%
    dplyr::slice_head(n = 25)
  
  scaled_feature_importance <- scaled_feature_importance %>%
    dplyr::mutate(feature = factor(feature, levels = rev(scaled_feature_importance$feature)))
  
  p <- ggplot(scaled_feature_importance, aes(x = feature, y = total_scaled_importance)) +
    geom_bar(stat = "identity", fill = "steelblue", alpha = 0.8) +
    coord_flip() +
    labs(
      title = "Normalized Scaled Feature Importance (Top 25 Features)",
      subtitle = "Importance normalized within cohort/model and scaled by model performance (MC-CV AUC)",
      x = "Feature",
      y = "Scaled Normalized Importance"
    ) +
    theme_minimal() +
    theme(
      plot.title = element_text(size = 14, face = "bold"),
      plot.subtitle = element_text(size = 11),
      axis.text.y = element_text(size = 9)
    )
  
  # Display the plot
  print(p)
  
  # Also save to summary/plots directory
  plot_dir_summary <- file.path(summary_dir, "plots")
  dir.create(plot_dir_summary, showWarnings = FALSE, recursive = TRUE)
  
  ggplot2::ggsave(file.path(plot_dir_summary, "scaled_feature_importance_bar_chart.png"), p,
                  width = 12, height = 10, dpi = 300)
  cat("\n✓ Saved normalized scaled feature importance bar chart to:", 
      file.path(plot_dir_summary, "scaled_feature_importance_bar_chart.png"), "\n")
  
} else {
  cat("Required files not found. Please run Sections 6.3-6.4 first to generate:\n")
  cat("  -", best_feat_path, "\n")
  cat("  -", metrics_path, "\n")
  if (!file.exists(best_feat_path)) {
    cat("\nNote: Feature importance extraction may need to be added to the MC-CV code.\n")
  }
}


In [ ]:
# 6.6 Save classification MC-CV results

# Base output directory (should already exist from Section 3)
output_dir_base <- here("cohort_analysis", "outputs", "classification")
dir.create(output_dir_base, recursive = TRUE, showWarnings = FALSE)

# Summary directory for combined cohort comparisons
summary_dir <- file.path(output_dir_base, "summary")
dir.create(summary_dir, recursive = TRUE, showWarnings = FALSE)

# Save combined MC-CV files (for cross-cohort comparison)
readr::write_csv(classification_mc_metrics,
                 file.path(summary_dir, "cohort_classification_metrics_mc_cv.csv"))

# Save cohort-specific MC-CV files
for (cohort_name in unique(classification_mc_metrics$Cohort)) {
  cohort_dir <- file.path(output_dir_base, cohort_name)
  dir.create(cohort_dir, recursive = TRUE, showWarnings = FALSE)
  
  # Save cohort-specific MC-CV metrics
  cohort_metrics_subset <- classification_mc_metrics %>% filter(Cohort == cohort_name)
  readr::write_csv(cohort_metrics_subset,
                   file.path(cohort_dir, "cohort_classification_metrics_mc_cv.csv"))
}

cat("\n✓ Saved classification MC-CV metrics:")
cat("\n  - Combined (summary):", summary_dir)
cat("\n  - Cohort-specific:", file.path(output_dir_base, "CHD"), "and", file.path(output_dir_base, "MC"), "\n")

# Display summary
classification_mc_metrics



## 6.7 Visualize Cohort Classification Results

Create visualization plots for cohort classification analysis comparing cohorts using the summary folder files.


In [ ]:
# 6.7 Visualize Cohort Classification Results

# Create visualization plots for cohort classification analysis
# This creates visualizations comparing cohorts using the summary folder files
# Visualizations are saved to summary/plots/ for cross-cohort comparisons

library(here)

# Source the visualization script
if (!file.exists(here("scripts", "R", "create_visualizations_cohort.R"))) {
  stop("Cannot find scripts/R/create_visualizations_cohort.R. Ensure scripts are in scripts/R/ directory.")
}
source(here("scripts", "R", "create_visualizations_cohort.R"))

# Run visualizations using the base output directory (script will look for summary/ folder)
# This creates:
# - Feature importance heatmap (cohort comparison)
# - AUC heatmap (cohort comparison)
# - Scaled feature importance bar chart
# - AUC table
# - Cohort clinical feature Sankey diagram
# - Scaled normalized feature importance Sankey diagram
run_visualizations(output_dir = output_dir_base)

cat("\n✓ Cohort classification visualizations saved to:", file.path(output_dir_base, "summary", "plots"), "\n")
cat("  - Feature importance heatmap\n")
cat("  - AUC heatmap\n")
cat("  - Scaled feature importance bar chart\n")
cat("  - AUC table\n")
cat("  - Cohort clinical feature Sankey diagram\n")
cat("  - Scaled normalized feature importance Sankey diagram\n")
